In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [5]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths

In [6]:
# import the necessary packages
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K

In [7]:
class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		# first set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# second set of CONV => RELU => POOL layers
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# third set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
              
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1000))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

In [53]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = 50
INIT_LR = 1e-3
BS = 64

In [92]:
# init the image suffix, yset, and image list
print("[INFO] loading images...")
suffix = '.jpg'
img_list = []
yset = []

[INFO] loading images...


In [93]:
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

In [94]:
# Testing Cells
csv_file = 'balthus.csv'

In [95]:
# use csv file to grab images/labels
df = pd.read_csv(csv_file)
nameCol = df['#img']
predCol = df['Style']

In [96]:
# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

In [97]:
# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

In [98]:
# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

In [99]:
label_yval

{'cityscape': 4,
 'genre painting': 3,
 'landscape': 7,
 'literary painting': 9,
 'nude painting (nu)': 5,
 'portrait': 2,
 'self-portrait': 6,
 'sketch and study': 8,
 'still life': 0,
 'symbolic painting': 1}

In [100]:
len(labels)

96

In [101]:
label_yval[labels[0]]

3

In [102]:
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])

In [103]:
len(yset)

96

In [104]:
# remove text and leave fabric cave number for labels and zero index
#for i in range (0,len(yset)):
#    yset[i] = int(re.sub("\D", "", yset[i]))
#    yset[i] = yset[i]-1

In [105]:
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./images3/", base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [106]:
Y = []
X = []

In [107]:
# Shuffle the data
p = np.random.permutation(len(yset))

In [108]:
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])

In [109]:
# split the test and training set 75:25
split = int(len(X)*(.75))
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]

In [110]:
# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [111]:
trainX.shape

(72, 28, 28, 3)

In [112]:
# parsed Y data containers
trainY = []
testY = []

In [113]:
# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,12)
testY = np_utils.to_categorical(ytest,12)

In [114]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

In [115]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=12)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

[INFO] compiling model...


In [116]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/50
2/1 [============================================================] - 2s 811ms/step - loss: 2.1788 - acc: 0.2442 - val_loss: 3.6280 - val_acc: 0.3333
Epoch 2/50
2/1 [============================================================] - 1s 383ms/step - loss: 2.3721 - acc: 0.4873 - val_loss: 2.2534 - val_acc: 0.2083
Epoch 3/50
2/1 [============================================================] - 1s 395ms/step - loss: 1.9650 - acc: 0.2303 - val_loss: 2.1848 - val_acc: 0.1250
Epoch 4/50
2/1 [============================================================] - 1s 391ms/step - loss: 2.0312 - acc: 0.1910 - val_loss: 2.1449 - val_acc: 0.3333
Epoch 5/50
2/1 [============================================================] - 1s 378ms/step - loss: 1.8074 - acc: 0.4184 - val_loss: 2.1874 - val_acc: 0.3333
Epoch 6/50
2/1 [============================================================] - 1s 388ms/step - loss: 1.9525 - acc: 0.3681 - val_loss: 2.3124 - val_acc: 0.3333
Epoch 7/50
2/

In [79]:
# plot the training loss and accuracy 
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Label Prediction")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('figure.png')

In [80]:
# Shows chart in a popup. Close to keep going
#chart = cv2.imread('figure.png',1)
#cv2.imshow('Results',chart)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [81]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/225.0

In [82]:
predictionsMade = []

In [83]:
preds = model.predict(img_check)

In [84]:
# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)

In [85]:
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 

In [86]:
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.8333333333333334


In [87]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

In [88]:
# Append the new column
df['#predictions'] = finalPred

In [91]:
# new file name and save
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)